In [1]:
import os, csv
from dotenv import load_dotenv
import time
from datetime import datetime
import schedule
import requests

import pandas as pd
import pandas_ta as ta
import numpy as np

from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.foreignexchange import ForeignExchange
load_dotenv()

True

In [2]:
API_KEYS = os.getenv('ALPHA_KEYS')
cc = ForeignExchange(key=API_KEYS)

LINE_TOKEN = os.getenv('LINE_TOKEN')
LINE_URL = 'https://notify-api.line.me/api/notify'
LINE_HEADERS = {'content-type':'application/x-www-form-urlencoded','Authorization':'Bearer ' + LINE_TOKEN }

ts = TimeSeries(key=API_KEYS, output_format='pandas')

In [ ]:
class RsiO2Notify:
    def __init__(self, params, ts):
        self.ts = ts
        self.pair = params['pair']
        self.tf = params['tf']
        self.order = {'action': None, 'open time': None, 'open': None, 'close time': None, \
                      'close': None, 'T/P': None, 'S/L': None,'result': None}
        
        self.current_action = 'close'
        self.atr_len = params['atr_len']
        self.rsi_len = params['rsi_len']
        self.ema_len = params['ema_len']
        
        self.overbought = params['overbought']
        self.oversold = params['oversold']
        self.pip_value = params['pip_value']
        self.rr = params['rr']
        
        self.line_url = params['line_url']
        self.line_token = params['line_token']
        self.line_headers = {'content-type':'application/x-www-form-urlencoded','Authorization':'Bearer ' + self.line_token }
        self.df = None
        
    def notifyMsg(self, *msg):
        r = requests.post(self.line_url, headers=self.line_headers, data={'message': msg[0]})
        print(r.text)
        
    def initial_bars(self):
        bars, _ = self.ts.get_intraday(symbol=self.pair,interval=self.tf, outputsize='full')
        df = pd.DataFrame({
            'timestamp': bars.index,
            'open': bars['1. open'],
            'high': bars['2. high'],
            'low': bars['3. low'],
            'close': bars['4. close'],
            'volumn': bars['5. volume']
        }) # convert alpha vantage data to use with strategy

        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df = df.iloc[::-1] # reverse row
        df.reset_index(drop=True, inplace=True)

        self.df = df
    
    def get_realtime(self):
        bars, _ = self.ts.get_intraday(symbol=self.pair,interval=self.tf, outputsize='compact')
        incoming_df = pd.DataFrame({
            'timestamp': bars.index,
            'open': bars['1. open'],
            'high': bars['2. high'],
            'low': bars['3. low'],
            'close': bars['4. close'],
            'volumn': bars['5. volume']
        }) # convert alpha vantage data to use with rsi strategy

        incoming_df['timestamp'] = pd.to_datetime(incoming_df['timestamp'], unit='ms')
     
        if (incoming_df['timestamp'].iloc[0] != self.df['timestamp'].iloc[-1]):
            self.df = self.df[:900].append(incoming_df.head(1), ignore_index = True)

        rsi_o2 = ta.Strategy(
            name = 'RSI 14 Days for opposite trade',
            description = 'RSI for alert',
            ta = [
                {'kind': 'ema', 'length': self.ema_len},
                {'kind': 'rsi', 'length': self.rsi_len},
                {'kind': 'atr', 'length': self.atr_len}
            ]
        )
        self.df.ta.strategy(rsi_o2)
    
        row = self.df.tail(1).to_dict('records')[0]
        
        if self.current_action == 'close' or self.order == {}:
            if (row['EMA_'+str(self.ema_len)] < row['low']) and (row['RSI_'+str(self.rsi_len)] > self.overbought):
#                 row['action'] = 'buy'
                self.order['action'] = 'buy'
                self.order['open time'] = row['timestamp']
                self.order['open'] = row['open']
                
                self.order['S/L'] = row['low']- row['ATRr_'+str(self.atr_len)]
                self.order['T/P'] = row['open'] + (self.rr * abs(row['open'] - self.order['S/L']))
                self.current_action = 'buy'
                self.notifyMsg(f"Buy {self.pair} RSI_O2\nprice: {str(self.order['open'])}\nTP: {str(self.order['T/P'])}\nSL: {str(self.order['S/L'])}")
                print(f"Buy {self.pair} \nopen: {str(self.order['open'])}\nclose: {str(self.order['close'])}\n " + \
                      f"ema: {str(row['EMA_'+str(self.ema_len)])}, rsi: {str(row['RSI_'+str(self.rsi_len)] )}")
                
            elif (row['EMA_'+str(self.ema_len)] > row['high']) and (row['RSI_'+str(self.rsi_len)] < self.oversold):   
#                 row['action'] = 'sell'
                self.order['action'] = 'sell'
                self.order['open time'] = row['timestamp']
                self.order['open'] = row['open']
                
                self.order['S/L'] = row['high'] + row['ATRr_'+str(self.atr_len)]
                self.order['T/P'] = row['open'] - (self.rr * abs(row['open'] - self.order['S/L']))
                self.current_action = 'sell'
                self.notifyMsg(f"Sell {self.pair} RSI_O2\nprice: {str(self.order['open'])}\nTP: {str(self.order['T/P'])}\nSL: {str(self.order['S/L'])}")
                print(f"Sell {self.pair} \nopen: {str(self.order['open'])}\nclose: {str(self.order['close'])}\n " + \
                      f"ema: {str(row['EMA_'+str(self.ema_len)])}, rsi: {str(row['RSI_'+str(self.rsi_len)] )}")

        if (self.current_action == 'buy'):
            if (row['low'] <= self.order['S/L']):
                self.order['result'] = 'S/L'
                self.order['close time'] = row['timestamp']
                self.order['close'] = self.order['S/L']
                
                self.current_action = 'close'
                self.order = {'action': None, 'open time': None, 'open': None, 'close time': None, \
                              'close': None, 'T/P': None, 'S/L': None,'result': None}
                self.notifyMsg(f"S/L Buy {self.pair} RSI_O2\nprice: {self.order['close']}")
                
            elif (row['high'] >= self.order['T/P']):
                self.order['result'] = 'T/P'
                self.order['close time'] = row['timestamp']
                self.order['close'] = self.order['T/P']
                
                self.current_action = 'close'
                self.order = {'action': None, 'open time': None, 'open': None, 'close time': None, \
                              'close': None, 'T/P': None, 'S/L': None,'result': None}
                self.notifyMsg(f"T/P Buy {self.pair} RSI_O2\nprice: {self.order['close']}")

        elif (self.current_action == 'sell'):
            if (row['high'] >= self.order['S/L']):
                self.order['result'] = 'S/L'
                self.order['close time'] = row['timestamp']
                self.order['close'] = self.order['S/L']
                
                self.current_action = 'close'
                self.notifyMsg(f"S/L Sell {self.pair} RSI_O2\nprice: {self.order['close']}")
                self.order = {'action': None, 'open time': None, 'open': None, 'close time': None, \
                              'close': None, 'T/P': None, 'S/L': None,'result': None}
                
            elif (row['low'] <= self.order['T/P']):
                self.order['result'] = 'T/P'
                self.order['close time'] = row['timestamp']
                self.order['close'] = self.order['T/P']
                
                self.current_action = 'close'
                self.notifyMsg(f"T/P Sell {self.pair} RSI_O2\nprice: {self.order['close']}")
                self.order = {'action': None, 'open time': None, 'open': None, 'close time': None, \
                              'close': None, 'T/P': None, 'S/L': None,'result': None}

In [4]:
eurusd = RsiO2Notify({
    'pair': 'EURUSD',
    'tf': '15min',
    'atr_len': 15,
    'rsi_len': 10,
    'ema_len': 200,
    'overbought': 76,
    'oversold': 24,
    'pip_value': 0.001,
    'rr': 1.5,
    'line_url': LINE_URL,
    'line_token': LINE_TOKEN,
}, ts)

gbpusd = RsiO2Notify({
    'pair': 'GBPUSD',
    'tf': '15min',
    'atr_len': 10,
    'rsi_len': 20,
    'ema_len': 200,
    'overbought': 70,
    'oversold': 30,
    'pip_value': 0.001,
    'rr': 1.5,
    'line_url': LINE_URL,
    'line_token': LINE_TOKEN,
}, ts)

usdjpy = RsiO2Notify({
    'pair': 'USDJPY',
    'tf': '15min',
    'atr_len': 17,
    'rsi_len': 12,
    'ema_len': 200,
    'overbought': 73,
    'oversold': 27,
    'pip_value': 0.001,
    'rr': 1.5,
    'line_url': LINE_URL,
    'line_token': LINE_TOKEN,
}, ts)

In [5]:
eurusd.initial_bars()

In [6]:
def get_all_realtime():
    eurusd.get_realtime()
    
schedule.every(60 * 15).seconds.do(get_all_realtime) 

Every 900 seconds do get_all_realtime() (last run: [never], next run: 2022-02-08 15:50:08)

In [ ]:
while True:
    schedule.run_pending()
    time.sleep(1)

{"status":200,"message":"ok"}
Sell EURUSD 
open: 1.1402
close: None
 ema: 1.1415469117542751, rsi: 23.309582319567635
{"status":200,"message":"ok"}
{"status":200,"message":"ok"}
Sell EURUSD 
open: 1.1407
close: None
 ema: 1.1415479067791507, rsi: 23.78581065023537
